This blog post describes how to use the bokeh plotting library to create some interactive plots

### Run boilerplate code

In [ ]:
%run genomeplot.ipynb

In [1]:
# output live to notebook
output_notebook()

### Load ag1k data

In [4]:
sys.path.insert(0, '../../../selection_paper/agam-report-base/src/python')
ag1k_dir = '/kwiat/vector/ag1000g/release'
from ag1k import phase1_ar3

In [5]:
phase1_ar3.init(os.path.join(ag1k_dir, 'phase1.AR3'))

In [6]:
chromosomes = "2R", "2L", "3R", "3L", "X"

### Define genome plot function

In [8]:
# Custom changes to axes that pertain to genome not data
# eg in this case move left chromosome arm axes to RHS
def custom_plot(chrom, subplot):
    if chrom.endswith("L"):
        subplot.yaxis.visible = False
        subplot.add_layout(LinearAxis(), 'right')

### Calculate $\theta$, $\Pi$, and Tajima's D

In [9]:
@functools.lru_cache()
def calculate_summary_stats(chrom, pop, window_size=100000):
    
    ix = phase1_ar3.df_samples.query("population == @pop").index
    accessibility = phase1_ar3.accessibility[chrom]["is_accessible"][:]
    
    pos = allel.SortedIndex(phase1_ar3.callset_pass[chrom]["variants/POS"][:])
    eqw = allel.equally_accessible_windows(accessibility, window_size)
    g = allel.GenotypeChunkedArray(phase1_ar3.callset_pass[chrom]["calldata/genotype"]).take(ix, axis=1)
    ac = g.count_alleles()
    
    theta, wins, nb, counts = allel.stats.windowed_watterson_theta(pos, ac, windows=eqw, is_accessible=accessibility)
    
    pi, wins, nb, counts = allel.stats.windowed_diversity(pos, ac, windows=eqw, is_accessible=accessibility)
    
    tajD, wins, counts = allel.stats.windowed_tajima_d(pos, ac, windows=eqw)
    
    df = pd.DataFrame.from_dict({"start": eqw[:, 0], 
                                 "stop": eqw[:, 1], 
                                 "diversity": pi, 
                                 "tajimaD": tajD, 
                                 "theta": theta})

    df["midpoint"] = eqw.mean(1)
    
    return df

In [10]:
stats = {c: calculate_summary_stats(chrom=c, pop="BFS", window_size=100000) for c in chromosomes}

### Use a .gff3 file to annotate above windows

In [11]:
gff3 = allel.FeatureTable.from_gff3(phase1_ar3.geneset_agamp42_fn, attributes=["ID"])

In [12]:
gff3

<FeatureTable shape=(175804,) dtype=(numpy.record, [('seqid', 'O'), ('source', 'O'), ('type', 'O'), ('start', '<i8'), ('end', '<i8'), ('score', '<f8'), ('strand', 'O'), ('phase', '<i8'), ('ID', 'O')])>
[('2L', 'VectorBase', 'contig',        1, 49364325, -1., '.', -1, '2L')
 ('2L', 'VectorBase', 'exon',   157348,   157623, -1., '-', -1, 'AGAP004677-RB-E4A')
 ('2L', 'VectorBase', 'exon',   157348,   157623, -1., '-', -1, 'AGAP004677-RB-E4B')
 ...,
 ('X', 'VectorBase', 'gene', 24338771, 24340371, -1., '+', -1, 'AGAP013609')
 ('X', 'VectorBase', 'rRNA', 24338771, 24340371, -1., '+', -1, 'AGAP013609-RA')
 ('Y_unplaced', 'VectorBase', 'contig',        1,   237045, -1., '.', -1, 'Y_unplaced')]

In [13]:
annotated_data = {}

# annotate these data
for chrom in chromosomes:

    d = stats[chrom].copy()
    
    # extract the relevant seq id and use pandas interval indexing
    features = pd.DataFrame(gff3.query("seqid == '{0}'".format(chrom)).values)
    features.index = pd.IntervalIndex.from_arrays(features.start, features.end, closed="both")

    # logic to extract relevant rows, filter by annot type, drop duplicates and join ID column
    # it would be slightly more efficient to do these both in a single call, but it's fast/readable so we'll let it slide
    d["gene"] = d.apply(
        lambda y: ", ".join(features.loc[[y.start, y.stop]].query("type == 'gene'").ID.drop_duplicates()), 1)

    annotated_data[chrom] = d

In [14]:
annotated_data["X"].head()

,diversity,start,stop,tajimaD,theta,midpoint,gene
0,0.001781,25,132324,-2.190824,0.005357,66174.5,AGAP000011
1,0.003892,132325,246994,-2.041654,0.010288,189659.5,"AGAP000011, AGAP000018"
2,0.005919,246995,368599,-2.176849,0.017548,307797.0,AGAP000018
3,0.006699,368600,487739,-2.297078,0.022272,428169.5,
4,0.009483,487740,625447,-2.252033,0.030146,556593.5,


In [15]:
def stat_plot(chrom, subplot, stat="diversity", label=None):
    
    source = ColumnDataSource(annotated_data[chrom])
    
    if label is None:
        label = stat
    
    hover = HoverTool(
        tooltips=[
            ("Position", "@start{0a.000}-@stop{0a.000}"),
            (label, "$y"),
            ("contig", chrom), 
            ("gene(s)", "@gene")],
        mode="mouse")
    
    subplot.add_tools(hover)
    
    subplot.circle("midpoint", 
                   stat,
                   source=source,
                   size=3, 
                   color="navy", 
                   alpha=0.5)

    subplot.yaxis[0].axis_label = label
    subplot.xaxis[0].axis_label = "Genomic Position (bp)"

In [27]:
gf = GenomePlot(fasta=phase1_ar3.genome_fn, contigs=("2R", "2L", "3R", "3L", "X"), layout="oo|ooo", pfunc=custom_plot)

In [28]:
gf.apply(stat_plot, stat="tajimaD", label="Tajima'sD")

In [29]:
gf.apply(stat_plot, stat="theta", label="\u0398W")

In [31]:
gf.apply(stat_plot, stat="diversity", label="\u03A0")